# Copyright

<PRE>
Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
</PRE>

# Introduction to deep learning-based classification and evaluation
Today, we will dive into deep learning classifiers, with an ephasis on evaluation methods. 


## 1. Activation functions for classification
First, lets have an intuation about the output of the final layers' activation functions. For binary and multilabel classification we use sigmoid, for multiclass classification softmax.

In [1]:
import numpy as np

def sigmoid(x):
  return 1/(1 + np.exp(-x))

a = np.linspace(-5,5,40)
print(a)
print(sigmoid(a))

[-5.         -4.74358974 -4.48717949 -4.23076923 -3.97435897 -3.71794872
 -3.46153846 -3.20512821 -2.94871795 -2.69230769 -2.43589744 -2.17948718
 -1.92307692 -1.66666667 -1.41025641 -1.15384615 -0.8974359  -0.64102564
 -0.38461538 -0.12820513  0.12820513  0.38461538  0.64102564  0.8974359
  1.15384615  1.41025641  1.66666667  1.92307692  2.17948718  2.43589744
  2.69230769  2.94871795  3.20512821  3.46153846  3.71794872  3.97435897
  4.23076923  4.48717949  4.74358974  5.        ]
[0.00669285 0.00863217 0.01112713 0.01433278 0.01844474 0.02370801
 0.03042661 0.03897319 0.04979714 0.06342879 0.08047598 0.10160773
 0.12751884 0.1588691  0.19619362 0.23978727 0.28957771 0.34501473
 0.40501421 0.46799255 0.53200745 0.59498579 0.65498527 0.71042229
 0.76021273 0.80380638 0.8411309  0.87248116 0.89839227 0.91952402
 0.93657121 0.95020286 0.96102681 0.96957339 0.97629199 0.98155526
 0.98566722 0.98887287 0.99136783 0.99330715]


In [2]:
def softmax(x):
  return(np.exp(x)/np.exp(x).sum())

pred1 = [100,10,1]
pred2 = [20,10,1]
pred3 = [2,1,1]
pred4 = [-10,0,10]

print(softmax(pred1), np.sum(pred1), np.sum(softmax(pred1)))
print(softmax(pred2), np.sum(pred2), np.sum(softmax(pred2)))
print(softmax(pred3), np.sum(pred3), np.sum(softmax(pred3)))
print(softmax(pred4), np.sum(pred4), np.sum(softmax(pred4)))

[1.00000000e+00 8.19401262e-40 1.01122149e-43] 111 1.0
[9.99954597e-01 4.53978684e-05 5.60254205e-09] 31 1.0
[0.57611688 0.21194156 0.21194156] 4 1.0
[2.06106005e-09 4.53978686e-05 9.99954600e-01] 0 0.9999999999999999


# 2. Model training
Let's train a simple MLP (Multi-Layer Perceptron) model on the CIFAR10 dataset and explore its capabilities.

In [3]:
# imports 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [4]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [5]:
flattened_dim = np.prod(X_train.shape[1:])

In [6]:
# reshape 3D tensors to 2D tensors
X_train = X_train.reshape(-1, flattened_dim)
X_test = X_test.reshape(-1, flattened_dim)

# it is in int8 format, the neural network requires float32
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [7]:
# train, valid, test split
train_ratio  = 0.8
train_length = X_train.shape[0]
train_split  = int(train_ratio*train_length)
X_valid, Y_valid = X_train[train_split:], Y_train[train_split:]
X_train, Y_train = X_train[:train_split], Y_train[:train_split]

In [8]:
# standardization
mean = X_train.mean(axis=0)
std  = X_train.std(axis=0)
X_train = (X_train-mean)/std
X_valid = (X_valid-mean)/std
X_test  = (X_test-mean)/std

In [9]:
# one-hot encoding
nb_classes = len(np.unique(Y_train))
Y_train = to_categorical(Y_train, nb_classes)
Y_valid = to_categorical(Y_valid, nb_classes)
Y_test  = to_categorical(Y_test, nb_classes)

In [10]:
# callbacks
tb = TensorBoard(log_dir='logs', histogram_freq=0, write_graph=0)
es = EarlyStopping(patience=5, restore_best_weights=True, verbose=1, monitor='val_accuracy')

In [11]:
# model definition with dropout
model = Sequential()
model.add(Dense(1024, activation='relu', kernel_initializer=HeNormal(), input_shape=(flattened_dim,)))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_initializer=HeNormal()))
model.add(Dense(nb_classes, activation='softmax'))

# loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# training
network_history = model.fit(X_train, Y_train, 
                            validation_data=(X_valid,Y_valid),
                            batch_size=128,                             
                            epochs=400000, 
                            verbose=1, 
                            callbacks=[tb,es])

Epoch 1/400000
313/313 [==============================] - 24s 70ms/step - loss: 1.9345 - accuracy: 0.3745 - val_loss: 1.6159 - val_accuracy: 0.4294
Epoch 2/400000
313/313 [==============================] - 21s 67ms/step - loss: 1.5353 - accuracy: 0.4559 - val_loss: 1.5280 - val_accuracy: 0.4628
Epoch 3/400000
313/313 [==============================] - 23s 74ms/step - loss: 1.4369 - accuracy: 0.4908 - val_loss: 1.4765 - val_accuracy: 0.4816
Epoch 4/400000
313/313 [==============================] - 24s 77ms/step - loss: 1.3646 - accuracy: 0.5141 - val_loss: 1.4698 - val_accuracy: 0.4857
Epoch 5/400000
313/313 [==============================] - 25s 80ms/step - loss: 1.3104 - accuracy: 0.5361 - val_loss: 1.4485 - val_accuracy: 0.4943
Epoch 6/400000
313/313 [==============================] - 22s 69ms/step - loss: 1.2613 - accuracy: 0.5530 - val_loss: 1.4468 - val_accuracy: 0.5002
Epoch 7/400000
313/313 [==============================] - 23s 73ms/step - loss: 1.2206 - accuracy: 0.5687 - val_

# 3. Classification evaluation on test data
Here, we will perform a detailed evaluation. Let's start by running an inference on the test data and get the predicted labels by argmax.


In [13]:
# imports 
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score, recall_score, precision_score, f1_score, roc_curve, auc
import seaborn as sns # for visualisation
import matplotlib.pyplot as plt # for visualisation
import matplotlib.colors as mcolors


In [14]:
probas = model.predict(X_test)
preds  = np.argmax(probas,axis=1)
Y_test_dense = np.argmax(Y_test, axis=1) # get the original dense labels of the test data

313/313 [==============================] - 3s 10ms/step


## 3.1. Confusion matrix

In [15]:
conf

NameError: name 'conf' is not defined

In [ ]:
conf = confusion_matrix(Y_test_dense, preds)
plt.figure(figsize=(7,6))
sns.heatmap(conf, fmt='d', annot=True, vmin=1, vmax=200)

Please, analyize the results - which classess are missclassified to other classes? See the [CIFAR10 class list](https://keras.io/api/datasets/cifar10/) on the Keras page. 

## 3.2. Accuracy, Precision, Recall, F1
We can calculate these values based on the confusion matrix, or separately with function calls, or we can display all of them with one function.

In [ ]:
print(accuracy_score(Y_test_dense, preds))
print(precision_score(Y_test_dense, preds, average='micro'))
print(precision_score(Y_test_dense, preds, average='macro'))
print(precision_score(Y_test_dense, preds, average='weighted'))
print(recall_score(Y_test_dense, preds, average='micro'))
print(recall_score(Y_test_dense, preds, average='macro'))
print(recall_score(Y_test_dense, preds, average='weighted'))
print(f1_score(Y_test_dense, preds, average='micro'))
print(f1_score(Y_test_dense, preds, average='macro'))
print(f1_score(Y_test_dense, preds, average='weighted'))


In [ ]:
print(classification_report(np.argmax(Y_test,axis=1),preds))

In [ ]:
print(balanced_accuracy_score(np.argmax(Y_test,axis=1),preds))

Indeed, it is the same as the standard accuracy score, because the test dataset is perfectly balanced (1000 samples per each class). 

## 3.3. ROC curve
ROC curves can be applied to binary classification problems, thus, we have to adjust the predictions of each class into a binary case.

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(np.unique(Y_test_dense))
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], probas[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=list(mcolors.TABLEAU_COLORS)[i], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()
# source: https://stackoverflow.com/questions/37017400/sklearn-metrics-roc-curve-for-multiclass-classification

## 3.4. Prediction probabilities
As a next step, it helps us to understand the capabilities of a classifier by analizing the histogram of the predicted probabilties. we wil show the idea for one class, however, it can be extended to all classes.

We will evaluate the probabilities in a one-vs-rest manner, which means that we will select one class, and merge all other classess.

In [ ]:
selected_class=7
Y_test_binary = Y_test_dense==selected_class
probas_binary = np.concatenate((np.sum(np.delete(probas,selected_class,axis=1),axis=1).reshape(-1,1), probas[:,selected_class].reshape(-1,1)), axis=1)

In [ ]:
TP=(Y_test_dense == selected_class) & (preds == selected_class)
TN=(Y_test_dense != selected_class) & (preds != selected_class)
FP=(Y_test_dense != selected_class) & (preds == selected_class)
FN=(Y_test_dense == selected_class) & (preds != selected_class)

sns.histplot(probas_binary[TP,1], kde=False, bins=40)
sns.histplot(probas_binary[FP,1], kde=False, bins=40, color='red')
sns.histplot(probas_binary[FN,1], kde=False, bins=40, color='yellow')


What does this figure mean? Think about it!

We can also display the probability distribution of the target class of the TN cases:

In [ ]:
sns.histplot(probas_binary[TN,1], kde=False, bins=40, color='green')


## 3.5. Exercise
Select and display 10-10 images that belong to class 7 and

* 3.5.1. the same class was (correctly) predicted, with the highest probabilities 
* 3.5.2. a different class was (incorrectly) predicted, with the highest probabilities
* 3.5.3. the same class was (correctly) predicted, with the lowest probabilities
* 3.5.4. a different class was (incorrectly) predicted, with the lowest probabilities
